In [1]:
import pandas as pd
import pickle

from dataset import NewsDataset

from smooth_gradient import SmoothGradient
from integrated_gradient import IntegratedGradient

from collections import defaultdict
import string

import torch
from torch import nn
from torch.utils.data import DataLoader
from transformers import DistilBertConfig, DistilBertTokenizer
from transformers import DebertaConfig

from IPython.display import display, HTML

from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding

2025-08-08 00:31:05.262911: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754613065.612843      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754613065.715016      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
from huggingface_hub import login

login(new_session=False, # Won’t request token if one is already saved on machine
write_permission=True, # Requires a token with write permission
token="", # The name of your token
add_to_git_credential=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in 'login': write_permission. Will not be supported from version '1.0'.

Fine-grained tokens added complexity to the permissions, making it irrelevant to check if a token has 'write' access.
  warnings.warn(message, FutureWarning)
Token has not been saved to git credential helper.


Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.


In [3]:
checkpoint = "microsoft/deberta-large-mnli"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/729 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.62G [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
config = DebertaConfig()
batch_size = 2
criterion = nn.CrossEntropyLoss()
config.max_position_embeddings

512

## Running on Dataset

In [5]:
from datasets import load_dataset, concatenate_datasets
import json
import ast

In [6]:
test_df4 = pd.read_csv('/kaggle/input/confer-extension/Updated_Part5_Dataset/Part5_Type4_final.csv',index_col=0)
test_df4

,premise,hypothesis,gold_label,trigger,SetID,UID,Trigger Words,premise_type
0,"If Alexander is an electrical technician, he'l...",Alexander has an insulation tester.,E,possessive,0,type4_possessive_00,his insulation tester,unrelated
1,"If Alexander is an electrical technician, he'l...",Alexander has an insulation tester.,E,possessive,0,type4_possessive_00,his insulation tester,somewhat_related
2,"If Alexander is an electrical technician, he'l...",Alexander has an insulation tester.,E,possessive,0,type4_possessive_00,his insulation tester,related
3,"If Ali is a firefighter, he'll borrow us his f...",Ali has a firehose.,E,possessive,1,type4_possessive_01,his firehose,somewhat_related
4,"If Ali is a firefighter, he'll pick his fireho...",Ali has a firehose.,E,possessive,1,type4_possessive_01,his firehose,unrelated
...,...,...,...,...,...,...,...,...
292,"If the girl is a yoga teacher, she'll grab her...",The girl has yoga pants.,E,possessive,97,type4_possessive_97,her yoga pants,unrelated
293,"If the girl is a yoga teacher, she'll use her ...",The girl has yoga pants.,E,possessive,97,type4_possessive_97,her yoga pants,related
294,"If the king is a teacher, he'll pack his suitc...",The king has a suitcase.,E,possessive,98,type4_possessive_98,his suitcase,unrelated
295,"If the king is a teacher, he'll pack his suitc...",The king has a suitcase.,E,possessive,98,type4_possessive_98,his suitcase,somewhat_related


In [7]:
test_df5a = pd.read_csv('/kaggle/input/confer-extension/Updated_Part5_Dataset/Part5_Type5A_final.csv',index_col=0)
test_df5a

,premise,hypothesis,gold_label,trigger,SetID,UID,Trigger Words,premise_type
0,"If Jessica attends a movie festival, she'll ne...",Jessica has watched a movie by Nolan before.,E,again,0,type5a_again_00,a movie festival,related
1,"If Jessica attends the conference, she'll neve...",Jessica has watched a movie by Nolan before.,E,again,0,type5a_again_00,attends the conference,unrelated
2,"If Emily finishes a mystery novel, she'll neve...",Emily has read a book by Dan Brown before.,E,again,1,type5a_again_01,finishes a mystery novel,related
3,"If Emily passes her driving test, she'll never...",Emily has read a book by Dan Brown before.,E,again,1,type5a_again_01,passes her driving test,unrelated
4,"If Scarlett bakes a Swedish pastry, she'll nev...",Scarlett has eaten a traditional Swedish dish ...,E,again,2,type5a_again_02,bakes a Swedish pastry,related
...,...,...,...,...,...,...,...,...
795,"If Margaret finishes work early, she'll never ...",Margaret has managed to keep a secret before.,E,again,397,type5a_again_397,finishes work early,unrelated
796,If Betty cherishes quiet time after missing th...,Betty has cherished a moment in peace before.,E,again,398,type5a_again_398,cherishes quiet time after,related
797,"If Betty misses the bus, she'll never cherish ...",Betty has cherished a moment in peace before.,E,again,398,type5a_again_398,misses the bus,unrelated
798,If Zoey holds a student responsible when drivi...,Zoey has held a student responsible for a mist...,E,again,399,type5a_again_399,holds a student responsible when driving,related


In [8]:
test_df5p = pd.read_csv('/kaggle/input/confer-extension/Updated_Part5_Dataset/Part5_Type5P_final.csv',index_col=0)
test_df5p

,premise,hypothesis,gold_label,trigger,SetID,UID,Trigger Words,premise_type
0,"If Jessica attends the conference, her colleag...",Jessica has a colleague.,E,possessive,0,type5p_possessive_00,her colleague,related
1,"If Jessica attends the conference, her colleag...",Jessica has a colleague.,E,possessive,0,type5p_possessive_00,her colleague,unrelated
2,"If Emily passes her driving test, her best fri...",Emily has a best friend.,E,possessive,1,type5p_possessive_01,her best friend,related
3,"If Emily passes her driving test, her best fri...",Emily has a best friend.,E,possessive,1,type5p_possessive_01,her best friend,unrelated
4,"If Scarlett finishes her chores early, her hus...",Scarlett has a husband.,E,possessive,2,type5p_possessive_02,her husband,related
...,...,...,...,...,...,...,...,...
795,"If Margaret finishes work early, her wife will...",Margaret has a wife.,E,possessive,397,type5p_possessive_397,his wife,unrelated
796,"If Betty misses the train, her brother will dr...",Betty has a brother.,E,possessive,398,type5p_possessive_398,her brother,related
797,"If Betty misses the train, her brother will ma...",Betty has a brother.,E,possessive,398,type5p_possessive_398,her brother,unrelated
798,"If Zoey decides to bike, her friends will meet...",Zoey has friends.,E,possessive,399,type5p_possessive_399,her friends,related


In [9]:
test_df = pd.concat([test_df4,test_df5a,test_df5p]).reset_index()
test_df

,index,premise,hypothesis,gold_label,trigger,SetID,UID,Trigger Words,premise_type
0,0,"If Alexander is an electrical technician, he'l...",Alexander has an insulation tester.,E,possessive,0,type4_possessive_00,his insulation tester,unrelated
1,1,"If Alexander is an electrical technician, he'l...",Alexander has an insulation tester.,E,possessive,0,type4_possessive_00,his insulation tester,somewhat_related
2,2,"If Alexander is an electrical technician, he'l...",Alexander has an insulation tester.,E,possessive,0,type4_possessive_00,his insulation tester,related
3,3,"If Ali is a firefighter, he'll borrow us his f...",Ali has a firehose.,E,possessive,1,type4_possessive_01,his firehose,somewhat_related
4,4,"If Ali is a firefighter, he'll pick his fireho...",Ali has a firehose.,E,possessive,1,type4_possessive_01,his firehose,unrelated
...,...,...,...,...,...,...,...,...,...
1892,795,"If Margaret finishes work early, her wife will...",Margaret has a wife.,E,possessive,397,type5p_possessive_397,his wife,unrelated
1893,796,"If Betty misses the train, her brother will dr...",Betty has a brother.,E,possessive,398,type5p_possessive_398,her brother,related
1894,797,"If Betty misses the train, her brother will ma...",Betty has a brother.,E,possessive,398,type5p_possessive_398,her brother,unrelated
1895,798,"If Zoey decides to bike, her friends will meet...",Zoey has friends.,E,possessive,399,type5p_possessive_399,her friends,related


In [10]:
sample_test_df = test_df5a.iloc[:4].reset_index()
sample_test_df

,index,premise,hypothesis,gold_label,trigger,SetID,UID,Trigger Words,premise_type
0,0,"If Jessica attends a movie festival, she'll ne...",Jessica has watched a movie by Nolan before.,E,again,0,type5a_again_00,a movie festival,related
1,1,"If Jessica attends the conference, she'll neve...",Jessica has watched a movie by Nolan before.,E,again,0,type5a_again_00,attends the conference,unrelated
2,2,"If Emily finishes a mystery novel, she'll neve...",Emily has read a book by Dan Brown before.,E,again,1,type5a_again_01,finishes a mystery novel,related
3,3,"If Emily passes her driving test, she'll never...",Emily has read a book by Dan Brown before.,E,again,1,type5a_again_01,passes her driving test,unrelated


In [11]:
possessive_trigger = False
test_df = sample_test_df
# test_df = test_df

In [12]:
test_example = [
    test_df['premise'], 
    test_df['hypothesis']
]

test_dataset = NewsDataset(
    data_list=test_example,
    tokenizer=tokenizer,
    max_length=256, 
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
)

In [13]:
integrated_grad = IntegratedGradient(
    model, 
    criterion, 
    tokenizer, 
    show_progress=False,
    # encoder="bert"
)
instances = integrated_grad.saliency_interpret(test_dataloader)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1830: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)


In [14]:
def reformat_output(instances):

    instances_word_gradients = []

    for instance in instances: 
        i = 0 # position counter 
        token_gradients = defaultdict(float) # dict to keep track of grad of each token     
        for token, grad in zip(instance['tokens'], instance['grad']):
            if token != '<unk>':
                token_gradients[(token,i)] += grad  # each token has its IG value added to the dict 
                i += 1
        
        token_gradients = dict(token_gradients)
        tokens = [tok[0] for tok in list(token_gradients.keys())]
        word_gradients = {}
    
        for i, token in enumerate(tokens):
            word = tokens[i]
            if word.startswith('Ġ'):
                word = word[1:]
            count = 1 
            word_gradient = token_gradients[(token,i)]
            if word in ['[SEP]','[CLS]']:
                 word_gradients[word] = word_gradient
                 continue # do not merge with next token
            while (i+1)<len(tokens) and tokens[i+1] not in string.punctuation and not tokens[i+1].startswith('Ġ') and not tokens[i+1] == '[SEP]':
                # part of same word, add gradients together and then average out 
                i += 1 # move to next word
                word += tokens[i] # concatente to restore word 
                word_gradient += token_gradients[(tokens[i],i)]
                count += 1
            # end of word 
            word_gradients[(word,i)] = word_gradient/count # averaging
        instances_word_gradients.append(word_gradients)
    
    return instances_word_gradients

In [15]:
def get_average_gradient(instances):
    # average gradient per token
    return [sum(instance['grad'])/len(instance['grad']) for instance in instances]

In [16]:
def compute_trigger_gradients(row):
    
    gradients = 0
    count = 0
    trigger_words = []

    # split phrases into individual words
    trigger_words = row['Trigger Words'].split()

    # get gradient of all words in this example
    word_gradients = row['word_gradients']
    word_gradients_words = [key[0] for key in list(word_gradients.keys())] # extracting word portion of keys
    
    word_index_mapping = {}
    for word, index in list(word_gradients.keys()):
        word_index_mapping.setdefault(word,index) # set the first value (premise value) as the index
    
    for word in trigger_words:
        if word in word_gradients_words:
            word_index = word_index_mapping[word]
            gradients += word_gradients[(word,word_index)]
            count += 1 
        elif word.lower() in word_gradients_words:
            word_index = word_index_mapping[word.lower()]
            gradients += word_gradients[(word.lower(),word_index)]
            count += 1 
        else:
            print(f"{row.name}: {word} not found!")

    # compute avg gradient of trigger tokens
    return gradients/count if count > 0 else 0

In [17]:
def get_predicted_label(instances):
    return [instance['label'] for instance in instances]

In [18]:
def compute_trigger_word_ranking(word_gradients,possessive_trigger):

    sorted_dict = dict(sorted(word_gradients.items(), key=lambda item: item[1],reverse=True))
    sorted_dict = {k: v for k,v in sorted_dict.items() if k[0] not in ['[CLS]','[SEP]'] and k[0] not in string.punctuation}
    word_gradients_df = pd.DataFrame.from_dict(sorted_dict, orient='index', columns=['Gradient'])
    word_gradients_df.index = pd.MultiIndex.from_tuples(word_gradients_df.index, names=['Word', 'Position'])
    word_gradients_df['Ranking'] = range(1, len(word_gradients_df) + 1)
    word_gradients_df['Percentile Ranking'] = word_gradients_df['Ranking'].apply(lambda x: x / len(word_gradients_df))

    word_gradients_words = [key[0] for key in list(word_gradients.keys())] # extracting word portion of keys 
    word_index_mapping = {}
    for word, index in list(word_gradients.keys()):
        word_index_mapping.setdefault(word,index) # set the first value (premise value) as the index

    key_trigger_word = ''
    if possessive_trigger:
        if 'his' in word_gradients_words:
            key_trigger_word = 'his'
        elif 'her' in word_gradients_words:
            key_trigger_word = 'her'
        elif 'their' in word_gradients_words:
            key_trigger_word = 'their'
    else:
        key_trigger_word = 'again'

    try:
        key_trigger_word_index = word_index_mapping[key_trigger_word]
        key_trigger_word_key = (f'{key_trigger_word}', key_trigger_word_index)    
        key_trigger_word_rank = word_gradients_df.loc[key_trigger_word_key]['Ranking']
        key_trigger_word_percent_rank = word_gradients_df.loc[key_trigger_word_key]['Percentile Ranking']
    except:
        key_trigger_word_rank = None 
        key_trigger_word_percent_rank = None
        print(f"Trigger word not found! - {key_trigger_word}")

    return pd.Series([key_trigger_word_rank, key_trigger_word_percent_rank])

In [19]:
test_df['predicted_label'] = get_predicted_label(instances)
test_df['predicted_label'] = test_df['predicted_label'].map({0:'C',1:'N',2:'E'})
test_df

,index,premise,hypothesis,gold_label,trigger,SetID,UID,Trigger Words,premise_type,predicted_label
0,0,"If Jessica attends a movie festival, she'll ne...",Jessica has watched a movie by Nolan before.,E,again,0,type5a_again_00,a movie festival,related,E
1,1,"If Jessica attends the conference, she'll neve...",Jessica has watched a movie by Nolan before.,E,again,0,type5a_again_00,attends the conference,unrelated,E
2,2,"If Emily finishes a mystery novel, she'll neve...",Emily has read a book by Dan Brown before.,E,again,1,type5a_again_01,finishes a mystery novel,related,E
3,3,"If Emily passes her driving test, she'll never...",Emily has read a book by Dan Brown before.,E,again,1,type5a_again_01,passes her driving test,unrelated,E


In [20]:
test_df.to_csv('part1_part5_deberta_results.csv')